In [1]:
import pickle

import numpy as np

from log_prob import log_prob

/Users/franceschi/mysims/DISKLAB/disklab/diskradial.py:18: UserWarning: numba not available, opacity calculation will be very slow
  warnings.warn('numba not available, opacity calculation will be very slow')


In [2]:
fname = "options.pickle"
with open(fname, "rb") as fb:
    options = pickle.load(fb)

ndim = 7

# param0 = 10 ** ((np.random.rand() - 0.5) * 4)
# param1 = np.random.rand(ndim - 3, 1)
# param2 = (np.random.rand() + 0.5) / 100
# param3 = (np.random.rand() - 0.5)
#
# p0 = np.vstack((param0, param1, param2, param3)).T[0]

p0 = [
    7.0,
    0.730,
    0.558,
    0.017,
    0.625,
    0.008,
    0.050,
]

In [3]:
prob, blob = log_prob(p0, options, debugging=True)
prob

/Users/franceschi/mysims/IMLup/IMLup_SPHERE_fit/helper_functions.py:67: UserWarning: Disk mass is unreasonably high: M_disk / Mstar = 0.3
  warnings.warn(f'Disk mass is unreasonably high: M_disk / Mstar = {d.mass/mstar:.2g}')


disk to star mass ratio = 0.3
Reading runs/test_1551644126/image.out
	 Reverting to standard azimuthal averaging; will ignore `unit` argument.
y_mm_sim: [           nan 1.43746591e-12 9.23110695e-13 6.09715154e-13
 4.42091807e-13 3.52687478e-13 2.95464945e-13 2.50023956e-13
 2.15742918e-13 1.86961980e-13 1.63536762e-13 1.45152647e-13
 1.30488476e-13 1.17620769e-13 1.06716354e-13 9.75479772e-14
 8.91967759e-14 8.18356175e-14 7.57464302e-14 7.06916411e-14
 6.64663306e-14 6.34701834e-14 6.11526899e-14 5.91721417e-14
 5.71326633e-14 5.49592465e-14 5.26432019e-14 5.03691770e-14
 4.82046479e-14 4.61694100e-14 4.42561731e-14 4.24093130e-14
 4.06408097e-14 3.89554268e-14 3.73228089e-14 3.57962022e-14
 3.43885674e-14 3.30639721e-14 3.18644729e-14 3.06803629e-14
 2.95432247e-14 2.84450768e-14 2.74351485e-14 2.64879172e-14
 2.55784721e-14 2.47547319e-14 2.39918181e-14 2.32194978e-14
 2.24745226e-14 2.17705121e-14 2.11117539e-14 2.04842126e-14
 1.98860020e-14 1.93192542e-14 1.87783567e-14 1.825762

/anaconda3/envs/dipsy/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3420: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/anaconda3/envs/dipsy/lib/python3.7/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/anaconda3/envs/dipsy/lib/python3.7/site-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims, where=where)
/anaconda3/envs/dipsy/lib/python3.7/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  subok=False)
/anaconda3/envs/dipsy/lib/python3.7/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Reading runs/test_1551644126/image_sca.out
	 Reverting to standard azimuthal averaging; will ignore `unit` argument.
	 Reverting to standard azimuthal averaging; will ignore `unit` argument.
	 Reverting to standard azimuthal averaging; will ignore `unit` argument.
	 Reverting to standard azimuthal averaging; will ignore `unit` argument.
	 Reverting to standard azimuthal averaging; will ignore `unit` argument.
y_sca_sim: [4.19906142 4.16910495 4.13461573 4.09086413 4.06937134 4.03457049
 3.99669053 3.93815085 3.88551778 3.84647182 3.82874891 3.79901793
 3.76174535 3.71128388 3.67588065 3.63382291 3.60073461 3.57428926
 3.53907529 3.4887781  3.4474271  3.42430257 3.40612165 3.38462667
 3.35900054 3.32909209 3.29583671 3.25736296 3.22918809 3.20084064
 3.18080078 3.15600125 3.12686584 3.09240762 3.0643378  3.03726413
 3.00706184 2.98379101 2.96265667 2.94879321 2.92303376 2.89665333
 2.87465363 2.8518634  2.82117896 2.79641148 2.77463516 2.75672713
 2.73365057 2.69953071 2.66281563 2.6241

nan

In [4]:
blob

3570723811